In [2]:
import os
import time

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

from collections import OrderedDict

from ncfModel import MovieNCF
from customDataset import MovieDataset

In [2]:
dataPath = os.path.join(os.getcwd(), 'ml-latest-small', 'movieLens_ratings2.csv')

rating_df = pd.read_csv(dataPath)

In [3]:
rating_df

,userId,movieId,rating,timestamp
0,1,1,5.0,964982703
1,1,6,5.0,964982224
2,1,47,5.0,964983815
3,1,50,5.0,964982931
4,1,70,4.0,964982400
...,...,...,...,...
64289,610,166528,5.0,1493879365
64290,610,166534,5.0,1493848402
64291,610,168248,5.0,1493850091
64292,610,168250,5.0,1494273047


In [46]:
cocktailDataPath = os.path.join(os.getcwd(), '..', '..', 'Dataset', 'PRADA_cocktail_reviews.json')
cocktailDf = pd.read_json(cocktailDataPath)

# 'rating'은 0.8 확률로 5.0, 0.15 확률로 4.0, 0.02 확률로 3.0, 0.015 확률로 2.0, 0.015 확률로 1.0
cocktailDf['rating'] = np.random.choice([5.0, 4.0, 3.0, 2.0, 1.0], size=len(cocktailDf), p=[0.8, 0.15, 0.02, 0.015, 0.015])

cocktailDf = cocktailDf[['User', 'Cocktail', 'rating']]
cocktailDf.columns =    ['userId', 'itemId', 'rating']

In [47]:
cocktailDf

,userId,itemId,rating
0,Ann G,2,5.0
1,debeca,2,5.0
2,GarbageCan,2,5.0
3,Justyna Gebela,2,4.0
4,Trish,2,5.0
...,...,...,...
5589,Heinrich45,10822,5.0
5590,DIELiz,10822,5.0
5591,Silviatempelmayr,10822,5.0
5592,snakeeleven,10187,5.0


In [48]:
# num_user_rating이 4개 이상인 유저만 사용
cocktailDf = cocktailDf.groupby('userId').filter(lambda x: len(x) >= 4)

# num_item_rating이 3개 이상인 아이템만 사용
cocktailDf = cocktailDf.groupby('itemId').filter(lambda x: len(x) >= 3)

cocktailDf



,userId,itemId,rating
4,Trish,2,5.0
8,Angela,2,5.0
24,Jillian,2,5.0
30,Lauren,2,4.0
32,Katie,2,5.0
...,...,...,...
5589,Heinrich45,10822,5.0
5590,DIELiz,10822,5.0
5591,Silviatempelmayr,10822,5.0
5592,snakeeleven,10187,5.0


In [7]:
# dataframe 저장
cocktailDf.to_csv(os.path.join(os.getcwd(), 'ml-latest-small', 'cocktail_ratings.csv'), index=False)

In [54]:
# userId, itemId를 1부터 시작하도록 변경
userIdMapper = {}
for new, old in enumerate(cocktailDf['userId'].unique()):
    userIdMapper[old] = new+1

itemIdMapper = {}
for new, old in enumerate(cocktailDf['itemId'].unique()):
    itemIdMapper[old] = new+1

cocktailDf['userId'] = cocktailDf['userId'].apply(lambda x: userIdMapper[x])
cocktailDf['itemId'] = cocktailDf['itemId'].apply(lambda x: itemIdMapper[x])



In [55]:
cocktailDf = cocktailDf[['userId', 'itemId', 'rating']]
cocktailDf

,userId,itemId,rating
4,1,1,5.0
8,2,1,5.0
24,3,1,5.0
30,4,1,4.0
32,5,1,5.0
...,...,...,...
5589,58,81,5.0
5590,48,81,5.0
5591,54,81,5.0
5592,52,42,5.0


In [11]:
# 모델 불러오기
MovieNCF_weight = torch.load(os.path.join(os.getcwd(), '..', '..', 'NCF_5_3_0.4_0.001_256.pth'))

for i in MovieNCF_weight:
    print(i)

user_embedding_GMF.weight
item_embedding_GMF.weight
user_embedding_MLP.weight
item_embedding_MLP.weight
MLP_layers.1.weight
MLP_layers.1.bias
MLP_layers.3.weight
MLP_layers.3.bias
MLP_layers.3.running_mean
MLP_layers.3.running_var
MLP_layers.3.num_batches_tracked
MLP_layers.5.weight
MLP_layers.5.bias
MLP_layers.7.weight
MLP_layers.7.bias
MLP_layers.7.running_mean
MLP_layers.7.running_var
MLP_layers.7.num_batches_tracked
MLP_layers.9.weight
MLP_layers.9.bias
MLP_layers.11.weight
MLP_layers.11.bias
MLP_layers.11.running_mean
MLP_layers.11.running_var
MLP_layers.11.num_batches_tracked
predict_layer.weight
predict_layer.bias


In [12]:
# MLP_layers만 추출
MLP_layers = [(layer, MovieNCF_weight[layer]) for layer in MovieNCF_weight if 'MLP_layers' in layer]

# OrderedDict로 변환
MLP_layers = OrderedDict(MLP_layers)

MLP_layers

OrderedDict([('MLP_layers.1.weight',
              tensor([[-0.2426, -0.0868,  0.1939, -0.2505,  0.1455,  0.1260,  0.2877,  0.2752,
                        0.1336, -0.3119, -0.2921, -0.1021,  0.0468,  0.1380,  0.2008, -0.0970,
                        0.1432, -0.1896, -0.1159, -0.0683, -0.2550, -0.2095,  0.2015,  0.0367,
                       -0.0453, -0.2157,  0.2895,  0.2249,  0.2088, -0.1926,  0.2383, -0.2670,
                       -0.2105,  0.2881, -0.0961,  0.0838, -0.1654, -0.0036, -0.2823, -0.2656],
                      [ 0.0451,  0.0547, -0.0459,  0.2917,  0.2793, -0.0379,  0.2319,  0.0914,
                       -0.1863,  0.2255,  0.0640,  0.0935, -0.0217, -0.2003,  0.0521,  0.0212,
                       -0.2757, -0.0885,  0.0539,  0.1835, -0.2700, -0.1245, -0.0110, -0.1378,
                       -0.2770, -0.2680, -0.0962,  0.1091,  0.2854,  0.1696,  0.0600, -0.3002,
                        0.1408,  0.0397, -0.2092, -0.0366,  0.0229,  0.3175, -0.1415, -0.2234],
           

In [13]:
import torch
import torch.nn as nn

class CocktailNCF(nn.Module):
    """
        :param num_users: number of users
        :param num_items: number of items
        :param num_factors: number of predictive factors
        :param dropout: dropout rate
        :param num_layers: number of layers in MLP model
        :param model: 'NCF'
        :param GMF: pretrained GMF model
        :param MLP: pretrained MLP model
    """
    def __init__(self, num_users, num_items, num_factors=5, dropout=0.04, num_layers=3, model=None, GMF=None, MLP=None):
        super(MovieNCF, self).__init__()

        self.num_factors = num_factors
        self.model = model
        self.GMF = GMF
        self.MLP = MLP

        self.dropout = dropout
        self.num_layers = num_layers
        
        
        self.user_embedding_GMF = nn.Embedding(num_users, num_factors)
        self.item_embedding_GMF = nn.Embedding(num_items, num_factors)
        self.user_embedding_MLP = nn.Embedding(num_users, num_factors*(2**(num_layers-1)))
        self.item_embedding_MLP = nn.Embedding(num_items, num_factors*(2**(num_layers-1)))
        

        MLP_modules = []

        for i in range(num_layers):
            input_size = num_factors * 2**(num_layers-i)
            MLP_modules.append(nn.Dropout(dropout))
            MLP_modules.append(nn.Linear(input_size, input_size//2))
            MLP_modules.append(nn.ReLU())
            MLP_modules.append(nn.BatchNorm1d(input_size//2))
        self.MLP_layers = nn.Sequential(*MLP_modules)

        predict_size = num_factors * 2

        self.predict_layer = nn.Linear(predict_size, 1)

        self._init_weight_()
        self.ReLU = nn.ReLU()
        self.Sigmoid = nn.Sigmoid()

    def _init_weight_(self):
        nn.init.normal_(self.user_embedding_GMF.weight, std=0.01)
        nn.init.normal_(self.item_embedding_GMF.weight, std=0.01)
        nn.init.normal_(self.user_embedding_MLP.weight, std=0.01)
        nn.init.normal_(self.item_embedding_MLP.weight, std=0.01)

        for layer in self.MLP_layers:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
        nn.init.kaiming_uniform_(self.predict_layer.weight, a=1, nonlinearity='sigmoid')

        for layer in self.modules():
            if isinstance(layer, nn.Linear) and layer.bias is not None:
                layer.bias.data.zero_()
        

    def forward(self, user, item):
        embedding_user_GMF = self.user_embedding_GMF(user)
        embedding_item_GMF = self.item_embedding_GMF(item)

        vector_GMF = torch.mul(embedding_user_GMF, embedding_item_GMF)

        embedding_user_MLP = self.user_embedding_MLP(user)
        embedding_item_MLP = self.item_embedding_MLP(item)

        interaction = torch.cat([embedding_user_MLP, embedding_item_MLP], -1)
        vector_MLP = self.MLP_layers(interaction)
        
        # 각 모델 결과에 가중치를 곱해준다.
        vector_GMF = vector_GMF * 0.5
        vector_MLP = vector_MLP * 0.5
        
        vector_concat = torch.cat([vector_GMF, vector_MLP], -1)

        prediction = self.predict_layer(vector_concat)
        prediction = self.Sigmoid(prediction)

        return prediction.view(-1)
        
    
    def predict(self, user, item):
        return self.forward(self.interaction2vec(item, user))
    
    def get_loss(self, user, item, rating):
        prediction = self.forward(self.interaction2vec(item, user))
        return nn.MSELoss()(prediction, rating)
    
    def get_topk(self, user, item, topk=10):
        prediction = self.forward(self.interaction2vec(item, user))
        return torch.topk(prediction, topk)
    
    def get_user_embedding(self, user):
        return self.user_embedding(user)
    
    def get_item_embedding(self, item):
        return self.item_embedding(item)
    
    def get_user_item_embedding(self, user, item):
        return self.user_embedding(user), self.item_embedding(item)



In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
train, test = train_test_split(cocktailDf, test_size=0.1, random_state=1234)

train, val = train_test_split(train, test_size=0.1, random_state=1234)

In [16]:
# 데이터 로더 정의
train = MovieDataset(train, is_training=True)
validation = MovieDataset(val, is_training=False)
test = MovieDataset(test, is_training=False)

train_loader = train.get_loader(32)
val_loader = validation.get_loader(32)
test_loader = test.get_loader(32)

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

GMF = None
MLP = None

num_users = cocktailDf['userId'].max()
num_items = num_items=cocktailDf['itemId'].max()

print("num_users: ", num_users)
print("num_items: ", num_items)

num_users:  86
num_items:  81


In [18]:
class args:
    lr = 0.001
    batch_size = 256
    epochs = 100
    num_factors = 5
    dropout = 0.04
    num_layers = 3
    model = 'NCF'
    GMF = None
    MLP = None
    topk = 10

In [19]:
model = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.02, num_layers=3, model='NCF', GMF=None, MLP=None)
model = model.to(device)

# model에 MLP_layers에 해당하는 weight를 로드
model.load_state_dict(MLP_layers, strict=False)

criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5) # Best

In [20]:
model_name = "CocktailNCF"+"_"+str(args.num_factors)+"_"+str(args.num_layers)+"_"+str(args.dropout)+"_"+str(args.lr)+"_"+str(args.batch_size)

In [21]:
### training
epochs = 100
best_rmse = 1000
for epoch in range(epochs):
    model.train() # enable dropout if used
    start = time.time()
    #train_loader.dataset.ng_sample()
    
    train_loss = 0

    for users, items, ratings in train_loader:
        users, items, ratings = users.cuda(), items.cuda(), ratings.cuda()
        
        model.zero_grad()
        prediction = model(users, items)
        loss = criterion(prediction, ratings)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
    train_loss /= len(train_loader)
    if (epoch+1) % 10 == 0:
        elapsed = time.time() - start
        print("The time elapse of epoch {:03d}".format(epoch) + " is: " + 
			time.strftime("%H: %M: %S", time.gmtime(elapsed)))
        print("Epoch : {}, Train Loss : {:.4f}".format(epoch+1, train_loss))
    #wandb.log({"train_loss": train_loss})

    val_loss = 0
    model.eval()
    for users, items, ratings in val_loader:
        with torch.no_grad():
            users, items, ratings = users.cuda(), items.cuda(), ratings.cuda()
            prediction = model(users, items)
            val_loss += criterion(prediction, ratings).item()
            
    val_loss /= len(val_loader)
    #wandb.log({"val_loss": val_loss})
    if (epoch+1) % 10 == 0:
        print("Epoch : {}, Val Loss : {:.4f}".format(epoch+1, val_loss))
    if val_loss < best_rmse:
        best_rmse = val_loss
        torch.save(model.state_dict(), model_name+".pth")
        print("Save model in file: {}.pth".format(model_name))

    #scheduler.step(metrics=val_loss)
    #scheduler.step()
print("Best RMSE: {:.4f}".format(best_rmse))

Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
The time elapse of epoch 009 is: 00: 00: 00
Epoch : 10, Train Loss : 0.4703
Epoch : 10, Val Loss : 0.5361
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
Save model in file: CocktailNCF_5_3_0.04_0.001_256.pth
The time elapse of epoch 019 is: 00: 00: 00
Epoch : 20, Train Loss : 0.3934
Epoch : 20, Val Loss : 0.5403
The time elapse of epoch 029 is: 00: 00: 00
Epoch : 30, Train Loss : 0.3818
Epoch : 30, Val Loss : 0.6270


Exception ignored in: <function _releaseLock at 0x7f44617b7040>
Traceback (most recent call last):
  File "/usr/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


The time elapse of epoch 039 is: 00: 00: 00
Epoch : 40, Train Loss : 0.3563
Epoch : 40, Val Loss : 0.5756
The time elapse of epoch 049 is: 00: 00: 00
Epoch : 50, Train Loss : 0.3215
Epoch : 50, Val Loss : 0.6400
The time elapse of epoch 059 is: 00: 00: 00
Epoch : 60, Train Loss : 0.3109
Epoch : 60, Val Loss : 0.5663
The time elapse of epoch 069 is: 00: 00: 00
Epoch : 70, Train Loss : 0.3019
Epoch : 70, Val Loss : 0.5862
The time elapse of epoch 079 is: 00: 00: 00
Epoch : 80, Train Loss : 0.2878
Epoch : 80, Val Loss : 0.5367
The time elapse of epoch 089 is: 00: 00: 00
Epoch : 90, Train Loss : 0.2956
Epoch : 90, Val Loss : 0.7855
The time elapse of epoch 099 is: 00: 00: 00
Epoch : 100, Train Loss : 0.2737
Epoch : 100, Val Loss : 0.5436
Best RMSE: 0.4567


In [ ]:
model.load_state_dict(torch.load(model_name+".pth"))
model.eval()
with torch.no_grad():
    test_loss = 0
    for users, items, ratings in test_loader:
        users, items, ratings = users.cuda(), items.cuda(), ratings.cuda()
        prediction = model(users, items)
        loss = criterion(prediction, ratings)
        
        test_loss += loss.item()
    test_loss /= len(test_loader)
    print("Test Loss : {:.4f}".format(test_loss))

    #wandb.log({"test_loss": test_loss})

    test_rmse = 0
    num_items = 0
    for users, items, ratings in test_loader:
        users, items, ratings = users.cuda(), items.cuda(), ratings.cuda()
        prediction = model(users, items)
        test_rmse += nn.MSELoss()(ratings.cpu(), prediction.cpu()).item()*len(ratings)
        num_items += len(ratings)
    test_rmse /= num_items
    test_rmse = np.sqrt(test_rmse)
    print("Test RMSE : {:.4f}".format(test_rmse))

Test Loss : 0.5609
Test RMSE : 0.7657


## 모델 성능측정

In [416]:
cocktailDataPath = os.path.join(os.getcwd(), '..', '..', 'Dataset', 'PRADA_cocktail_reviews.json')
cocktailDf = pd.read_json(cocktailDataPath)

# 'rating'은 0.8 확률로 5.0, 0.15 확률로 4.0, 0.02 확률로 3.0, 0.015 확률로 2.0, 0.015 확률로 1.0
cocktailDf['rating'] = np.random.choice([5.0, 4.0, 3.0, 2.0, 1.0], size=len(cocktailDf), p=[0.8, 0.15, 0.02, 0.015, 0.015])

cocktailDf = cocktailDf[['User', 'Cocktail', 'rating']]
cocktailDf.columns =    ['userId', 'itemId', 'rating']

In [417]:
# num_user_rating이 4개 이상인 유저만 사용
cocktailDf = cocktailDf.groupby('userId').filter(lambda x: len(x) >= 4)

# num_item_rating이 3개 이상인 아이템만 사용
cocktailDf = cocktailDf.groupby('itemId').filter(lambda x: len(x) >= 3)


# userId, itemId를 1부터 시작하도록 변경
userIdMapper = {}
for new, old in enumerate(cocktailDf['userId'].unique()):
    userIdMapper[old] = new+1

itemIdMapper = {}
for new, old in enumerate(cocktailDf['itemId'].unique()):
    itemIdMapper[old] = new+1

cocktailDf['userId'] = cocktailDf['userId'].apply(lambda x: userIdMapper[x])
cocktailDf['itemId'] = cocktailDf['itemId'].apply(lambda x: itemIdMapper[x])


cocktailDf = cocktailDf[['userId', 'itemId', 'rating']]
cocktailDf

,userId,itemId,rating
4,1,1,5.0
8,2,1,5.0
24,3,1,5.0
30,4,1,4.0
32,5,1,5.0
...,...,...,...
5589,58,81,5.0
5590,48,81,4.0
5591,54,81,5.0
5592,52,42,4.0


In [418]:
train, test = train_test_split(cocktailDf, test_size=0.1, random_state=42)

In [419]:
test_user = np.array(test['userId']-1)
test_item = np.array(test['itemId']-1)
test_rating = np.array(test['rating'])

print(test_user)
print(test_item)
print(test_rating)

[ 6 72 25 69 53  4 46 12  6 45 53  6 63 79 75 64 59 78 58 58 67  6  6 57
 30  6 47 70  5 72 26  6  3  6 50 45 16 47 51  7 62 33  6 72 76  6  6 47
 53 63  6 79 79 53 70  6 36 20 47 33 55 28  7 12 15 58  6 74 33  6 55 60
  6  6 39 46 58 58 83  6 85 46 51 76 47 16 62 46 60 45 54  6 50 80  6 55
 57 53 20 53 27 64  4 37  6 54 10 46  6 53]
[ 1 23  3 43 27  3 53  2  3 46 42 13 34 66 76 10 23 32  2 50 74  3 17 12
  3  3 78 27 10 51  3 21  3  3 76 57  9 66 49 46 44  3  1  1 28 16 17 65
 28 69  1 69 36 36 40  3  3  3 47 22 46  3 17 10  1 46  3 78 21 22  2 11
  3 13 17 80 77 10 46  1 55 24 34 44 41  3 73 75 72 12 60 21 40 76  3 41
 57 51 10  2 22 40  1 17 10  9 22 34 10 65]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5. 5. 5. 3. 5. 5. 4. 5.
 4. 5. 5. 5. 5. 5. 5. 4. 4. 5. 5. 5. 5. 4. 5. 4. 5. 4. 4. 5. 2. 5. 5. 4.
 5. 4. 4. 5. 5. 3. 5. 4. 5. 5. 5. 5. 5. 2. 1. 4. 5. 5. 5. 4. 4. 5. 4. 4.
 5. 5. 4. 5. 4. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 4. 5. 5. 5.
 5. 5. 5. 5. 4. 5. 4. 5. 4. 5. 5. 5.

In [420]:
num_users = len(cocktailDf['userId'].unique())
num_items = len(cocktailDf['itemId'].unique())

print(num_users, num_items)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

86 81


In [421]:
# 모델 8개 생성
datapath = os.path.join(os.path.curdir, '..', '..')


Pure32 = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.04, num_layers=3, model='NCF', GMF=None, MLP=None)
Pure32.load_state_dict(torch.load(os.path.join(datapath, 'NCF_NoPretrain_5_3_0.4_0.001_32.pth')))
Pure64 = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.04, num_layers=3, model='NCF', GMF=None, MLP=None)
Pure64.load_state_dict(torch.load(os.path.join(datapath, 'NCF_NoPretrain_5_3_0.4_0.001_64.pth')))
Pure128 = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.04, num_layers=3, model='NCF', GMF=None, MLP=None)
Pure128.load_state_dict(torch.load(os.path.join(datapath, 'NCF_NoPretrain_5_3_0.4_0.001_128.pth')))
Pure256 = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.04, num_layers=3, model='NCF', GMF=None, MLP=None)
Pure256.load_state_dict(torch.load(os.path.join(datapath, 'NCF_NoPretrain_5_3_0.4_0.001_256.pth')))

Pretrained32 = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.04, num_layers=3, model='NCF', GMF=None, MLP=None)
Pretrained32.load_state_dict(torch.load(os.path.join(datapath, 'NCF_Pretrained_5_3_0.4_0.001_32.pth')))
Pretrained64 = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.04, num_layers=3, model='NCF', GMF=None, MLP=None)
Pretrained64.load_state_dict(torch.load(os.path.join(datapath, 'NCF_Pretrained_5_3_0.4_0.001_64.pth')))
Pretrained128 = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.04, num_layers=3, model='NCF', GMF=None, MLP=None)
Pretrained128.load_state_dict(torch.load(os.path.join(datapath, 'NCF_Pretrained_5_3_0.4_0.001_128.pth')))
Pretrained256 = MovieNCF(num_users=num_users, num_items=num_items, num_factors=5, dropout=0.04, num_layers=3, model='NCF', GMF=None, MLP=None)
Pretrained256.load_state_dict(torch.load(os.path.join(datapath, 'NCF_Pretrained_5_3_0.4_0.001_256.pth')))



<All keys matched successfully>

In [422]:
# 테스트 데이터 prediction 출력
models = [Pure32, Pure64, Pure128, Pure256, Pretrained32, Pretrained64, Pretrained128, Pretrained256]
predictions = []
for model in models:
    model.to(device)
    model.eval()
    with torch.no_grad():
        prediction = model(torch.tensor(test_user).cuda(), torch.tensor(test_item).cuda())
        predictions.append(prediction.cpu().numpy())

In [423]:
print(test_rating)

[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5. 5. 5. 3. 5. 5. 4. 5.
 4. 5. 5. 5. 5. 5. 5. 4. 4. 5. 5. 5. 5. 4. 5. 4. 5. 4. 4. 5. 2. 5. 5. 4.
 5. 4. 4. 5. 5. 3. 5. 4. 5. 5. 5. 5. 5. 2. 1. 4. 5. 5. 5. 4. 4. 5. 4. 4.
 5. 5. 4. 5. 4. 5. 5. 5. 5. 5. 4. 5. 5. 5. 5. 5. 5. 5. 5. 5. 4. 5. 5. 5.
 5. 5. 5. 5. 4. 5. 4. 5. 4. 5. 5. 5. 5. 5.]


In [424]:
from sklearn.metrics import mean_squared_error
# RMSE 측정
for prediction in predictions:
    print(np.sqrt(mean_squared_error(test_rating, prediction)))


0.7046902638966279
0.748187629877932
0.7358519869756502
0.7334286266498466
0.728065160970204
0.7153440931795271
0.7369627132019795
0.7214720430684788


In [425]:
test_rating.shape
predictions[0].shape

(110,)

In [429]:
# R2 score 측정
from sklearn.metrics import r2_score
for prediction in predictions:
    print(r2_score(test_rating, prediction))

-0.027307104320646536
-0.15804329406369022
-0.12017192263733167
-0.11280601123486278
-0.09658991815433193
-0.058604509647249836
-0.12355614495901523
-0.07681913114135797


In [427]:
# MAE 측정
from sklearn.metrics import mean_absolute_error
for prediction in predictions:
    print(mean_absolute_error(test_rating, prediction))

0.48512051755731755
0.46344472061504016
0.5042399037968028
0.5135777256705544
0.4991095152768222
0.5177122939716686
0.5392832387577403
0.5165198759599166


In [428]:
num_users

86

In [411]:
# 모델별로 모든 유저와 아이템에 대한 예측값을 출력
# 예측값을 저장할 2차원 배열 생성
# 8개 모델, 86명 유저, 81개 아이템
all_predictions = np.zeros((8, 86, 81))
for i, model in enumerate(models):
    #all_predictions[i] = model(torch.tensor(np.arange(86)).cuda(), torch.tensor(np.arange(81)).cuda()).cpu().numpy()
    for j in range(num_users):
        for k in range(num_items):
            all_predictions[i][j][k] = model(torch.tensor([j]).cuda(), torch.tensor([k]).cuda()).cpu().detach().numpy()
    #print(model(torch.tensor([0]).cuda(), torch.tensor([0]).cuda()))

for i in range(8):
    print(all_predictions[i])


[[4.69253016 4.59713078 4.76891899 ... 4.69564438 4.69564629 4.69563818]
 [4.69134569 4.60487747 4.72068739 ... 4.69562149 4.6956892  4.69565153]
 [4.69489241 4.63167524 4.68767977 ... 4.69565678 4.6956315  4.69564056]
 ...
 [4.70218325 4.6107254  4.79802275 ... 4.69538784 4.69343519 4.6956625 ]
 [4.69552374 4.62594128 4.74216604 ... 4.69565582 4.69561911 4.69567728]
 [4.69562197 4.63955736 4.72262764 ... 4.69566631 4.69563675 4.69564247]]
[[4.84600067 4.84332371 4.87580204 ... 4.84170246 4.82654285 4.87864161]
 [4.68005466 4.21727943 4.64270449 ... 4.67726755 4.51702023 4.61834002]
 [4.85768032 4.83807087 4.91003609 ... 4.86129427 4.85142326 4.89951849]
 ...
 [4.85300112 4.76384497 4.93392324 ... 4.8681469  4.81421947 4.86901283]
 [4.91593075 4.8111639  4.90227795 ... 4.84519672 4.86761284 4.95303392]
 [4.81639194 4.81748009 4.91806889 ... 4.81959677 4.84695625 4.85674953]]
[[4.76194286 4.48977852 4.85185242 ... 4.88886738 4.6154871  4.7179656 ]
 [4.8251977  4.68156767 4.69840717 ... 

In [412]:
# cocktailDf에 중복 열 제거
cocktailDf = cocktailDf.drop_duplicates(['userId', 'itemId'], keep='first')

In [413]:
# 리뷰가 5개 이상인 유저들의 리뷰를 모두 출력
target_users = []
hit = 0
hit_rates = []

for prediction in all_predictions:
    hit = 0
    total = 0
    for i in range(num_users):
        if len(cocktailDf[cocktailDf['userId']==i]) >= 5:
            user_id = i
            true_top5 = cocktailDf[cocktailDf['userId']==i].sort_values(by=['rating'], ascending=False).head(5)

            prediction_top5 = np.argsort(prediction[i])[::-1][:5]
            intersection = np.intersect1d(true_top5['itemId'].values, prediction_top5)
            if len(intersection) > 0:
                hit += 1
    hit_rates.append(hit/num_users)

for i in range(8):
    print(hit_rates[i])

0.20930232558139536
0.16279069767441862
0.1511627906976744
0.1744186046511628
0.16279069767441862
0.13953488372093023
0.11627906976744186
0.16279069767441862


In [414]:
# rate hit rate 측정
hit_rates = []
for prediction in predictions:
    ones, ones_hit = 0, 0
    twos, twos_hit = 0, 0
    threes, threes_hit = 0, 0
    fours, fours_hit = 0, 0
    fives, fives_hit = 0, 0
    
    for i in range(len(test_rating)):
        #print(prediction[i])
        # 반올림
        rounded_rating = round(prediction[i])
        #print(rounded_rating)
        if test_rating[i] == 1:
            ones += 1
            if rounded_rating == 1:
                ones_hit += 1

        elif test_rating[i] == 2:
            twos += 1
            if rounded_rating == 2:
                twos_hit += 1

        elif test_rating[i] == 3:
            threes += 1
            if rounded_rating == 3:
                threes_hit += 1

        elif test_rating[i] == 4:
            fours += 1
            if rounded_rating == 4:
                fours_hit += 1

        elif test_rating[i] == 5:
            fives += 1
            if rounded_rating == 5:
                fives_hit += 1

    if ones != 0:
        ones_hit_rates = ones_hit/ones
    else:
        ones_hit_rates = None
    if twos != 0:
        twos_hit_rates = twos_hit/twos
    else:
        twos_hit_rates = None
    if threes != 0:
        threes_hit_rates = threes_hit/threes
    else:
        threes_hit_rates = None
    if fours != 0:
        fours_hit_rates = fours_hit/fours
    else:
        fours_hit_rates = None
    if fives != 0:
        fives_hit_rates = fives_hit/fives
    else:
        fives_hit_rates = None

    # print(ones, ones_hit, ones_hit_rates)
    # print(twos, twos_hit, twos_hit_rates)
    # print(threes, threes_hit, threes_hit_rates)
    # print(fours, fours_hit, fours_hit_rates)
    # print(fives, fives_hit, fives_hit_rates)


    hit_rates.append([ones_hit_rates, twos_hit_rates, threes_hit_rates, fours_hit_rates, fives_hit_rates])

for hr in hit_rates:
    print(hr)
    

[0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 0.0, 0.0, 0.125, 0.9195402298850575]
[0.0, 0.0, 0.0, 0.1875, 0.8505747126436781]
[0.0, 0.0, 0.0, 0.1875, 0.7816091954022989]
[0.0, 0.0, 0.0, 0.1875, 0.8045977011494253]
[0.0, 0.0, 0.0, 0.0625, 0.8620689655172413]
[0.0, 0.0, 0.0, 0.1875, 0.8160919540229885]
[0.0, 0.0, 0.0, 0.3125, 0.7816091954022989]


In [415]:
#CUDA_LAUNCH_BLOCKING=1
# TORCH_USE_CUDA_DSA

TORCH_CUDA_ALLOCATOR=device